In [1]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 
%pip install geopy
%pip install requests


import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

%pip install folium 

print("Libraries imported.")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [10]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [11]:
soup = BeautifulSoup(data, 'html.parser')

In [12]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [13]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [14]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


In [20]:
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


In [21]:
toronto_dropNotAssigned = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)

toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()


,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


In [22]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,\nM3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,\nM3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,\nM3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,\nM3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,\nM3GNot assigned


In [23]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood


In [24]:
toronto_df_grouped.shape

(20, 3)